In [1]:
import os
import sys
import matplotlib
import matplotlib.pyplot as plt
import dask.array as da
import dask.dataframe as dd
import xarray as xr
from xnemogcm import open_domain_cfg, get_metrics
import xgcm
import cartopy.crs as ccrs
import cmocean
import numpy as np
from scipy.stats import linregress
import datetime
import pandas as pd
import plots_spatial as pltspat
# Add SouthernDemons library to PATH
sys.path.append(os.path.abspath("../lib/"))
from teos_ten import teos_sigma0
import datesandtime

# Subdomain information (As inputted into TRACMASS, note non-pythonic indexing)
imindom = 1
imaxdom = 1440
jmindom = 1
jmaxdom = 400
kmindom = 1
kmaxdom = 75

# Location of the TRACMASS run
data_dir = os.path.abspath("/gws/nopw/j04/bas_pog/astyles/ORCA025_fwd/")

# Location of the OUTPUT directory created when running SouthernDemons executable
out_dir = os.path.abspath(data_dir + "/OUTPUT.ORCA025_fwd_extra/")
model_data = os.path.abspath(" /gws/nopw/j04/bas_pog/astyles/DRAKKAR_SET_for_Zhenya/month/" )   #ORCA025-N06_1999m03W.nc
# Location of masks and grid information for the model
grid_path = os.path.abspath("/gws/nopw/j04/bas_pog/astyles/ORCA025_fwd/topo/" )
grid_files = ['mask.nc','mesh_hgr.nc','mesh_zgr.nc']

#cal_months = ["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]

# Use dask to load the tabulated data lazily 
#df_ini = dd.read_parquet(out_dir + f"/df_ini.combined.parquet")
#df_out = dd.read_parquet(out_dir + f"/df_out.combined.parquet")
df_vent = dd.read_parquet(out_dir + f"/df_vent.parquet")
ds_domain = open_domain_cfg( datadir=grid_path, files = grid_files )

<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.fl

In [2]:
from pathlib import Path
from os import listdir

from xnemogcm import open_domain_cfg, open_nemo, process_nemo, open_namelist, open_nemo_and_domain_cfg
from xnemogcm import __version__ as xnemogcm_version

In [3]:
nemo = open_nemo(domcfg=ds_domain,files=['/gws/nopw/j04/bas_pog/astyles/DRAKKAR_SET_for_Zhenya/month/ORCA025-N06_1984m09T.nc'])


In [58]:
mld =nemo.mldr10_1


In [59]:
type(mld)

xarray.core.dataarray.DataArray

In [60]:
mldnp = mld.values

In [61]:
mask = np.where(mldnp>10)

In [62]:
nemo

<xarray.Dataset> Size: 1GB
Dimensions:               (t: 1, z_c: 75, y_c: 1021, x_c: 1442, axis_nbounds: 2)
Coordinates:
    time_centered         (t) datetime64[ns] 8B dask.array<chunksize=(1,), meta=np.ndarray>
  * t                     (t) datetime64[ns] 8B 1984-09-16
  * x_c                   (x_c) int64 12kB 0 1 2 3 4 ... 1438 1439 1440 1441
  * y_c                   (y_c) int64 8kB 0 1 2 3 4 ... 1016 1017 1018 1019 1020
    gdept_1d              (z_c) float64 600B dask.array<chunksize=(16,), meta=np.ndarray>
  * z_c                   (z_c) int64 600B 0 1 2 3 4 5 6 ... 69 70 71 72 73 74
    glamt                 (y_c, x_c) float32 6MB dask.array<chunksize=(128, 91), meta=np.ndarray>
    gphit                 (y_c, x_c) float32 6MB dask.array<chunksize=(128, 91), meta=np.ndarray>
    gdept_0               (z_c, y_c, x_c) float32 442MB dask.array<chunksize=(5, 128, 91), meta=np.ndarray>
Dimensions without coordinates: axis_nbounds
Data variables: (12/15)
    thetao                (t, z_c, y_c, x_c) float32 442MB dask.array<chunksize=(1, 1, 171, 206), meta=np.ndarray>
    time_centered_bounds  (t, axis_nbounds) datetime64[ns] 16B dask.array<chunksize=(1, 2), meta=np.ndarray>
    t_bounds              (t, axis_nbounds) datetime64[ns] 16B dask.array<chunksize=(1, 2), meta=np.ndarray>
    so                    (t, z_c, y_c, x_c) float32 442MB dask.array<chunksize=(1, 1, 171, 206), meta=np.ndarray>
    tos                   (t, y_c, x_c) float32 6MB dask.array<chunksize=(1, 171, 206), meta=np.ndarray>
    tossq                 (t, y_c, x_c) float32 6MB dask.array<chunksize=(1, 171, 206), meta=np.ndarray>
    ...                    ...
    wfo                   (t, y_c, x_c) float32 6MB dask.array<chunksize=(1, 171, 206), meta=np.ndarray>
    rsntds                (t, y_c, x_c) float32 6MB dask.array<chunksize=(1, 171, 206), meta=np.ndarray>
    tohfls                (t, y_c, x_c) float32 6MB dask.array<chunksize=(1, 171, 206), meta=np.ndarray>
    taum                  (t, y_c, x_c) float32 6MB dask.array<chunksize=(1, 171, 206), meta=np.ndarray>
    mldkz5                (t, y_c, x_c) float32 6MB dask.array<chunksize=(1, 171, 206), meta=np.ndarray>
    mldr10_1              (t, y_c, x_c) float32 6MB dask.array<chunksize=(1, 171, 206), meta=np.ndarray>
Attributes:
    name:         NEMO dataset
    description:  Ocean grid variables, set on the proper positions
    title:        Ocean grid variables
    Conventions:  CF-1.5
    production:   An IPSL model
    timeStamp:    2016-Apr-28 01:28:41 BST

In [63]:
mask

(array([0, 0, 0, ..., 0, 0, 0], shape=(723598,)),
 array([   1,    1,    1, ..., 1020, 1020, 1020], shape=(723598,)),
 array([ 361,  362,  363, ..., 1170, 1171, 1172], shape=(723598,)))

In [64]:
mldnp[0][361][1]

np.float32(40.333878)

In [65]:
value = nemo['mldr10_1'].sel(x_c=1400, y_c=1001)

In [66]:
value.values

array([nan], dtype=float32)

In [75]:
########## try for 2011 (small data)



In [117]:
df = df_vent[['year_o', 'month_o','binnedx_o','binnedy_o']]

In [118]:
df_group = df.groupby(['year_o', 'month_o','binnedx_o','binnedy_o'])

In [119]:
vol = df_group.sum('subvol_o')

In [120]:
interest = df[(df['year_o']==2011) & (df_vent['month_o'] ==2)]

In [121]:
#interest.head()

In [122]:
import dask.array as da
import time
t1= time.time()
# Assuming xs and ys are Dask arrays of valid indices
xs = interest.binnedx_o.to_dask_array().compute()
ys = interest.binnedy_o.to_dask_array().compute()
print(time.time()-t1)
value = nemo['mldr10_1'].isel(x_c=('points',xs), y_c=('points',ys))
print(time.time()-t1)
val_arr =value.data.squeeze()
value_series = dd.from_dask_array(val_arr, columns=["mld"])




15.086942672729492
15.091717720031738


In [123]:
interest['mld'] = value_series['mld'].compute()

In [124]:
#####PROBLEM THAT INDEXES DON'T MATCH
interest.head(20)

,year_o,month_o,binnedx_o,binnedy_o,mld
40355,2011,2,1149,156,NaN
50049,2011,2,334,204,NaN
130258,2011,2,854,324,NaN
60370,2011,2,316,166,NaN
217754,2011,2,1264,362,NaN
157649,2011,2,46,217,NaN
191567,2011,2,320,162,NaN
29327,2011,2,1223,353,NaN
121763,2011,2,544,180,NaN
11052,2011,2,218,157,NaN


In [125]:
print(interest.index.compute())  # Compute to see actual values
print(value_series.index.compute())

Index([ 40355, 104893,  32979,  29327, 135850, 170098,  18698, 113685,  50439,
       102172,
       ...
        38917, 145153, 176142,  44031,  72439,  39800, 138983, 137286, 132024,
       109842],
      dtype='int64', length=1115)
RangeIndex(start=0, stop=1115, step=1)


In [114]:
value_series['mld'].compute()

0       79.318886
1       73.038963
2       63.049660
3        3.994571
4       52.711395
          ...    
1110    44.192093
1111    34.285549
1112    49.711159
1113    53.707481
1114    57.090797
Name: mld, Length: 1115, dtype: float32

In [40]:
#### try to do it row by row
def monthstochar2(x):
    if x<10:
        strx = f'0{int(x)}'
    else:
        strx = f'{int(x)}' 
    return strx 

def find_mld (x,y,month,year=2011 ):
    month = monthstochar2(month)
    #print(f'DRAKKAR_SET_for_Zhenya/month/ORCA025-N06_{year}m{month}T.nc')
    nemo = open_nemo(domcfg=ds_domain,files=[f'/gws/nopw/j04/bas_pog/astyles/DRAKKAR_SET_for_Zhenya/month/ORCA025-N06_{year}m{month}T.nc'])
    print(type(x))
    
    value = nemo['mldr10_1'].sel(x_c=x, y_c=y)[0]    
    return value
    

In [43]:
val =find_mld (5,10,11,1993)

<class 'int'>


In [6]:
type(val)

xarray.core.dataarray.DataArray

In [54]:
df_interest = df_vent[(df_vent['year_o']==2011) & (df_vent['month_o'] ==2)]

In [55]:
import time
t1 = time.time()

df_interest['mld'] = df_interest.apply(
    lambda row: find_mld(row['binnedx_o'], row['binnedy_o'], row['month_o'], year=2011 ),
    axis=1, meta = ("mld", float) 
)
print(time.time()-t1)

2.433736562728882


In [56]:
df_interest.mld.compute()

KeyboardInterrupt: 

In [88]:
import time
t1 = time.time()
df_interest['mld'].head()
print(time.time() - t1)

KeyboardInterrupt: 

In [77]:
df_interest.head(10)

,ntraj_o,x_o,y_o,z_o,subvol_o,time_o,boxface_o,temp_o,sal_o,density_o,...,binnedy_i,binnedz_i,binnedx_o,binnedy_o,binnedz_o,nd_bin_ini,sf_zint,bathy_depth_i,bathy_depth_o,mld
476,227383,27.99,219.70,20.75,9.979545e+08,8.936395e+08,0,1.39,33.95,27.05,...,378,50,28,220,21,87,-126.164564,3687.088867,1785.798340,[<xarray.DataArray 'mldr10_1' ()> Size: 4B\nda...
2473,197262,465.74,128.95,21.71,9.979558e+08,8.942443e+08,0,-0.83,34.08,27.27,...,345,51,466,129,22,87,-102.484791,4759.973145,3552.920410,[<xarray.DataArray 'mldr10_1' ()> Size: 4B\nda...
2474,556748,16.17,205.91,21.13,9.641345e+08,8.942443e+08,0,0.65,33.99,27.12,...,225,38,16,206,21,108,35.248671,5177.729004,1121.924683,[<xarray.DataArray 'mldr10_1' ()> Size: 4B\nda...
3858,525642,334.00,207.19,20.76,9.754793e+08,8.926034e+08,1,2.50,33.96,26.97,...,295,45,334,207,21,80,-116.438150,4587.725440,3119.088623,[<xarray.DataArray 'mldr10_1' ()> Size: 4B\nda...
5262,383068,961.15,184.27,20.72,9.827383e+08,8.942443e+08,0,0.38,34.04,27.19,...,218,37,961,184,21,106,37.329259,4781.000977,1450.950499,[<xarray.DataArray 'mldr10_1' ()> Size: 4B\nda...
5761,80601,1178.55,144.64,21.00,9.721324e+08,8.940303e+08,6,-0.53,34.21,27.37,...,254,47,1179,145,21,102,5.439574,3270.617188,4268.395020,[<xarray.DataArray 'mldr10_1' ()> Size: 4B\nda...
5762,433060,1009.24,201.77,21.57,9.757728e+08,8.938123e+08,0,1.77,33.96,27.03,...,209,40,1009,202,22,113,40.349038,5097.219238,1723.279776,[<xarray.DataArray 'mldr10_1' ()> Size: 4B\nda...
7725,453506,1042.17,128.89,23.00,9.479292e+08,8.937035e+08,6,-0.61,34.20,27.36,...,157,44,1042,129,23,112,38.707426,4946.178223,4734.460938,[<xarray.DataArray 'mldr10_1' ()> Size: 4B\nda...
8321,499296,1409.90,158.76,19.00,9.855268e+08,8.922024e+08,6,-0.05,34.00,27.17,...,284,48,1410,159,19,83,-111.852986,4914.387207,3928.540039,[<xarray.DataArray 'mldr10_1' ()> Size: 4B\nda...
8322,58527,413.64,83.36,24.68,9.033503e+08,8.929483e+08,0,-1.27,34.08,27.29,...,107,45,414,83,25,101,7.374135,2950.185791,2065.952393,[<xarray.DataArray 'mldr10_1' ()> Size: 4B\nda...


In [29]:
nemo

NameError: name 'nemo' is not defined